# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
#!python create_tables.py

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [5]:
#get_files('data/song_data')

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files('data/song_data')

In [7]:
filepath = song_files[0]
filepath

'/home/workspace/data/song_data/A/B/B/TRABBLU128F93349CF.json'

In [8]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNNKDK1187B98BBD5,45.80726,Zagreb Croatia,15.9676,Jinx,407.37914,1,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
df[['song_id','title','artist_id','year','duration']]

,song_id,title,artist_id,year,duration
0,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),ARNNKDK1187B98BBD5,0,407.37914


In [10]:
song_data = df[['song_id','title','artist_id','year','duration']].values.tolist()[0]
song_data

['SOFNOQK12AB01840FC',
 'Kutt Free (DJ Volume Remix)',
 'ARNNKDK1187B98BBD5',
 0,
 407.37914]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [11]:
#from sql_queries import *
song_table_insert

'\nINSERT INTO songs (song_id, title, artist_id, year, duration) VALUES (%s, %s, %s, %s, %s)\n'

In [12]:
cur.execute(song_table_insert, song_data)
conn.commit()

In [13]:
print("\nTable: songs\n")
try: 
    cur.execute("SELECT * FROM songs WHERE artist_id = 'ARNNKDK1187B98BBD5' LIMIT 5;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()


Table: songs

('SOFNOQK12AB01840FC', 'Kutt Free (DJ Volume Remix)', 'ARNNKDK1187B98BBD5', 0, 407.37914)
('SOFNOQK12AB01840FC', 'Kutt Free (DJ Volume Remix)', 'ARNNKDK1187B98BBD5', 0, 407.37914)


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [14]:
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNNKDK1187B98BBD5,45.80726,Zagreb Croatia,15.9676,Jinx,407.37914,1,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),0


In [15]:
df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARNNKDK1187B98BBD5,Jinx,Zagreb Croatia,45.80726,15.9676


In [16]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values.tolist()[0]
artist_data

['ARNNKDK1187B98BBD5', 'Jinx', 'Zagreb Croatia', 45.80726, 15.967600000000001]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [17]:
artist_table_insert

'\nINSERT INTO artists (artist_id, name, location, lattitude, longitude) VALUES (%s, %s, %s, %s, %s)\n'

In [18]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

In [19]:
print("\nTable: artists\n")
try: 
    cur.execute("SELECT * FROM artists LIMIT 5;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()


Table: artists

('ARNNKDK1187B98BBD5', 'Jinx', 'Zagreb Croatia', 45.80726, 15.9676)
('ARNNKDK1187B98BBD5', 'Jinx', 'Zagreb Croatia', 45.80726, 15.9676)


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [20]:
log_files = get_files('data/log_data')

In [21]:
filepath = log_files[0]
filepath

'/home/workspace/data/log_data/2018/11/2018-11-29-events.json'

In [22]:
df = pd.read_json(filepath, lines=True)
#df.head()
df[df.page != 'NextSong'].head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
31,None,Logged In,Layla,F,114,Griffin,NaN,paid,"Lake Havasu City-Kingman, AZ",GET,Home,1.541057e+12,984,None,200,1543455764796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
43,None,Logged In,Theodore,M,0,Smith,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540306e+12,990,None,200,1543465425796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,52
50,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,1053,None,200,1543483675796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
58,None,Logged In,Celeste,F,0,Williams,NaN,free,"Klamath Falls, OR",GET,Home,1.541078e+12,947,None,200,1543499081796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
59,None,Logged In,Samuel,M,0,Gonzalez,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540493e+12,1001,None,200,1543499104796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [23]:
df = df[df.page == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [24]:
df.ts = pd.to_datetime(df.ts, unit='ms')

In [25]:
df.ts = pd.to_datetime(df.ts, unit='ms')
t = df.ts
t.head()

0   2018-11-29 00:00:57.796
1   2018-11-29 00:01:30.796
2   2018-11-29 00:04:01.796
3   2018-11-29 00:04:55.796
4   2018-11-29 00:07:13.796
Name: ts, dtype: datetime64[ns]

In [26]:
time_data = list(zip(t.values, t.dt.hour.values, t.dt.day.values, t.dt.week.values, \
                     t.dt.month.values, t.dt.year.values, t.dt.weekday.values))
column_labels = ['timestamp', 'hour', 'day', 'week of year', 'month', 'year', 'weekday']

In [27]:
time_df = pd.DataFrame(time_data, columns=column_labels)
time_df.head(5)

,timestamp,hour,day,week of year,month,year,weekday
0,2018-11-29 00:00:57.796,0,29,48,11,2018,3
1,2018-11-29 00:01:30.796,0,29,48,11,2018,3
2,2018-11-29 00:04:01.796,0,29,48,11,2018,3
3,2018-11-29 00:04:55.796,0,29,48,11,2018,3
4,2018-11-29 00:07:13.796,0,29,48,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [28]:
time_table_insert

'\nINSERT INTO time (start_time, hour, day, week, month, year, weekday) VALUES (%s, %s, %s, %s, %s, %s, %s)\n'

In [29]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [30]:
print("\nTable: time\n")
try: 
    cur.execute("SELECT * FROM time LIMIT 5;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()


Table: time

(datetime.datetime(2018, 11, 29, 0, 0, 57, 796000), 0, 29, 48, 11, 2018, 3)
(datetime.datetime(2018, 11, 29, 0, 1, 30, 796000), 0, 29, 48, 11, 2018, 3)
(datetime.datetime(2018, 11, 29, 0, 4, 1, 796000), 0, 29, 48, 11, 2018, 3)
(datetime.datetime(2018, 11, 29, 0, 4, 55, 796000), 0, 29, 48, 11, 2018, 3)
(datetime.datetime(2018, 11, 29, 0, 7, 13, 796000), 0, 29, 48, 11, 2018, 3)


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [31]:
df.head(5)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,2018-11-29 00:00:57.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,2018-11-29 00:01:30.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,2018-11-29 00:04:01.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,2018-11-29 00:04:55.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,2018-11-29 00:07:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [32]:
df[['userId','firstName','lastName','gender','level']].head(5)

,userId,firstName,lastName,gender,level
0,73,Jacob,Klein,M,paid
1,24,Layla,Griffin,F,paid
2,24,Layla,Griffin,F,paid
3,73,Jacob,Klein,M,paid
4,24,Layla,Griffin,F,paid


In [51]:
user_df = df[['userId','firstName','lastName','gender','level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [34]:
user_table_insert

'\nINSERT INTO users (user_id, first_name, last_name, gender, level) VALUES (%s, %s, %s, %s, %s)\n'

In [35]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

In [36]:
print("\nTable: users\n")
try: 
    cur.execute("SELECT * FROM users LIMIT 5;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()


Table: users

(73, 'Jacob', 'Klein', 'M', 'paid')
(24, 'Layla', 'Griffin', 'F', 'paid')
(24, 'Layla', 'Griffin', 'F', 'paid')
(73, 'Jacob', 'Klein', 'M', 'paid')
(24, 'Layla', 'Griffin', 'F', 'paid')


## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [53]:
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,2018-11-29 00:00:57.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [38]:
song_select

'\nSELECT s.song_id, \n       s.artist_id\n  FROM songs s\n  JOIN artists a\n    ON s.artist_id = a.artist_id\n WHERE s.title = %s\n       AND a.name = %s\n       AND s.duration = %s\n'

In [39]:
songplay_table_insert

'\nINSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)\n'

In [54]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

In [55]:
print("\nTable: songplays\n")
try: 
    cur.execute("SELECT * FROM songplays LIMIT 5;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()


Table: songplays

(1, datetime.datetime(2018, 11, 29, 0, 0, 57, 796000), 73, 'paid', None, None, 954, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
(2, datetime.datetime(2018, 11, 29, 0, 1, 30, 796000), 24, 'paid', None, None, 984, 'Lake Havasu City-Kingman, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')
(3, datetime.datetime(2018, 11, 29, 0, 4, 1, 796000), 24, 'paid', None, None, 984, 'Lake Havasu City-Kingman, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')
(4, datetime.datetime(2018, 11, 29, 0, 4, 55, 796000), 73, 'paid', None, None, 954, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
(5, datetime.datetime(2018, 

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.